In [132]:
import sys
from copy import deepcopy
import re
from typing import List, Tuple
from collections import Counter
import matplotlib.pyplot as plt

In [133]:
sys.path.append("../")

In [134]:
file_path = 'input_ag.txt'

In [135]:
description = []
with open(file_path, 'r') as file:
    for line in file:
        description.append(line.strip('\n'))

In [136]:
def separate_map_and_instructions(data_list):
    """
    Separates the map and instructions from the provided data list.
    """
    # Find the index of the blank line
    blank_line_index = data_list.index('')
    # Slice the list to get map lines and instructions
    map_lines = data_list[:blank_line_index]
    instructions_list = data_list[blank_line_index + 1:]
    # Join instructions if there are multiple lines
    instructions = ''.join(instructions_list).strip('\n')

    return map_lines, instructions

# Use the function to separate map and instructions
map_section, instructions_section = separate_map_and_instructions(description)

# Display the separated sections
print("**Map:**")
print("```\n" + "\n".join(map_section) + "\n```")

print("\n**Instructions:**")
print("```\n" + instructions_section + "\n```")

**Map:**
```
##################################################
#....OOO.#.OO.O.#O..OO.......#...O..O..O....O....#
#.O....O#.....#...O#.O..O.....OOO...O...##.OO..O.#
#..OO#O..O...OO.OO.#.OO..OO....OOO..OO...OO#OO...#
###O#O.OO.O.#.O.O...#.O.O...#OO......O.#OO....O..#
#.O.OOO.........O.....OOO..O#O....#O........O.O..#
#....O.OOO.O##O..O.OOO.O.....#....O....O....O.O#O#
#OO...#O.O..O..OO...O..#.O.O....OO..O.O...OO..O#.#
#O...O.#....................O...#..O.....O.OO...##
#..O.O...#.#.OO.....OO...O.O...O.O...OO.O.OO...O##
#.......O...O.....O..O...##..#O.#O.O.O..#......#O#
#.O.......#.O.O...O..O........#..O.OO.O#.O......O#
#OOO.O.#.O...O..O#O#.#..O.....O.#O...OO........O.#
#O.#..O#.O.#OO.O...O.O....O....####.O.O.O.#.OO#..#
#..#.O...#.#O.O.#OO...O.........#...........O.O..#
#..#...O..O.##O#........#..OO.....O......O...#...#
#..OO.O.O..#.O#O..#........#..O..OO#...O..#..#...#
#.O......#..O....OO..#....##OO..#.OO#O#O..O....O.#
#.....O.O...#O.O..OOO..#O...O.O...O.........OO#..#
#.OO......O.#..#O.

In [137]:
example_map = [
    "##########",
    "#..O..O.O#",
    "#......O.#",
    "#.OO..O.O#",
    "#..O@..O.#",
    "#O#..O...#",
    "#O..O..O.#",
    "#.OO.O.OO#",
    "#....O...#",
    "##########"
]

example_map = [list(row) for row in example_map]

example_instructions = [
'<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^',
'vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v',
'><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<',
'<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^',
'^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><',
'^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^',
'>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^',
'<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>',
'^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>',
'v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^',]
example_instructions = ''.join(example_instructions).strip()

In [138]:
example_instructions

'<vv>^<v^>v>^vv^v>v<>v^v<v<^vv<<<^><<><>>v<vvv<>^v^>^<<<><<v<<<v^vv^v>^vvv<<^>^v^^><<>>><>^<<><^vv^^<>vvv<>><^^v>^>vv<>v<<<<v<^v>^<^^>>>^<v<v><>vv>v^v^<>><>>>><^^>vv>v<^^^>>v^v^<^^>v^^>v^<^v>v<>>v^v^<v>v^^<^^vv<<<v<^>>^^^^>>>v^<>vvv^><v<<<>^^^vv^<vvv>^>v<^^^^v<>^>vvvv><>>v^<<^^^^^^><^><>>><>^^<<^^v>>><^<v>^<vv>>v>>>^v><>^v><<<<v>>v<v<v>vvv>^<><<>^><^>><>^v<><^vvv<^^<><v<<<<<><^v<<<><<<^^<v<^^^><^>>^<v^><<<^>>^v<v^v<v^>^>>^v>vv>^<<^v<>><<><<v<<v><>v<^vv<<<>^^v^>^^>>><<^v>>v^v><^^>>^<>vv^<><^^>^^^<><vvvvv^v<v<<>^v<v>v<<^><<><<><<<^^<<<^<<>><<><^^^>^^<>^>v<>^^>vv<^v^v<vv>^<><v<^v>^^^>>>^^vvv^>vvv<>>>^<^>>>>>^<<^v>^vvv<>^<><<v>v^^>>><<^^<>>^v^<v^vv<>v^<<>^<^v^v><^<<<><<^<v><v<>vv>>v><v^<vv<>v^<<^'

In [139]:
def find_robot_direction(dir_str):
    if dir_str == '^':
        return (-1,0)
    elif dir_str == '>':
        return (0,1)
    elif dir_str == 'v':
        return (1,0)
    elif dir_str == '<':
        return (0,-1)
    else:
        raise ValueError('Invalid direction')

In [140]:
def find_starting_position(map):
    starting_position = []
    for i in range(len(map)):
        for j in range(len(map[0])):
            if map[i][j] == '@':
                starting_position = (i, j)
    return starting_position

In [150]:
def move_robot(current_position, current_direction, map_grid):
    """
    Moves the robot on the map based on the current direction.
    If the robot encounters an 'O', it attempts to push it recursively until
    it either hits a wall ('#') or finds an empty space ('.') to move the 'O's.
    """
    def push_O(position, direction):
        """
        Recursively pushes 'O's in the given direction.
        """
        next_pos = (position[0] + direction[0], position[1] + direction[1])
        try:
            value = map_grid[next_pos[0]][next_pos[1]]
        except IndexError:
            # Out of bounds; treat as a wall
            return False

        if value == '#':
            # Cannot push 'O' into a wall
            return False
        elif value == 'O':
            # Attempt to push the next 'O'
            if not push_O(next_pos, direction):
                return False
            # Move the next 'O' forward
            map_grid[next_pos[0]][next_pos[1]] = 'O'
            map_grid[position[0]][position[1]] = '.'
            return True
        elif value == '.':
            # Move 'O' into the empty space
            map_grid[next_pos[0]][next_pos[1]] = 'O'
            map_grid[position[0]][position[1]] = '.'
            return True
        else:
            # Encountered an unexpected character; cannot push
            return False

    # Calculate the new virtual position based on direction
    new_virtual_position = (
        current_position[0] + current_direction[0],
        current_position[1] + current_direction[1]
    )

    try:
        new_virtual_value = map_grid[new_virtual_position[0]][new_virtual_position[1]]
    except IndexError:
        # If the new position is out of bounds, treat it as a wall
        return current_position, map_grid

    if new_virtual_value == '#':
        # Cannot move into a wall
        return current_position, map_grid
    elif new_virtual_value == 'O':
        # Attempt to push the 'O' in the current direction
        if push_O(new_virtual_position, current_direction):
            # If successful, move the robot into the 'O's previous position
            map_grid[current_position[0]][current_position[1]] = '.'  # Clear current position
            map_grid[new_virtual_position[0]][new_virtual_position[1]] = '@'  # Move robot
            current_position = new_virtual_position
        else:
            # Push failed; robot cannot move
            return current_position, map_grid
    elif new_virtual_value == '.':
        # Move robot into the empty space
        map_grid[current_position[0]][current_position[1]] = '.'  # Clear current position
        map_grid[new_virtual_position[0]][new_virtual_position[1]] = '@'  # Move robot
        current_position = new_virtual_position
    else:
        # Encountered an unexpected character; do nothing
        return current_position, map_grid

    return current_position, map_grid

def compute_gps_sum(map_grid):
    """
    Calculates the sum of GPS coordinates for all boxes ('O') on the map.
    """
    gps_sum = 0
    for row_idx, row in enumerate(map_grid):
        for col_idx, cell in enumerate(row):
            if cell == 'O':
                gps = 100 * row_idx + col_idx
                gps_sum += gps
    return gps_sum

def pretty_print_map(map_grid):
    for row in map_grid:
        print(''.join(row))


In [151]:
example_modified_map = deepcopy(example_map)
starting_position = find_starting_position(example_modified_map)
current_position = starting_position
example_modified_map[current_position[0]][current_position[1]] = '.'
for direction in example_instructions:
    current_position, map = move_robot(current_position, find_robot_direction(direction), example_modified_map)

In [152]:
map[current_position[0]][current_position[1]] = '@'
map

[['#', '#', '#', '#', '#', '#', '#', '#', '#', '#'],
 ['#', '.', 'O', '.', 'O', '.', 'O', 'O', 'O', '#'],
 ['#', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
 ['#', 'O', 'O', '.', '.', '.', '.', '.', '.', '#'],
 ['#', 'O', 'O', '@', '.', '.', '.', '.', '.', '#'],
 ['#', 'O', '#', '.', '.', '.', '.', '.', 'O', '#'],
 ['#', 'O', '.', '.', '.', '.', '.', 'O', 'O', '#'],
 ['#', 'O', '.', '.', '.', '.', '.', 'O', 'O', '#'],
 ['#', 'O', 'O', '.', '.', '.', '.', 'O', 'O', '#'],
 ['#', '#', '#', '#', '#', '#', '#', '#', '#', '#']]

In [153]:
print(compute_gps_sum(map))

10092


In [157]:
real_map = [list(row) for row in map_section]
instructions = list(instructions_section)

In [158]:
real_modified_map = deepcopy(real_map)
starting_position = find_starting_position(real_modified_map)
current_position = starting_position
real_modified_map[current_position[0]][current_position[1]] = '.'
for direction in instructions:
    current_position, map = move_robot(current_position, find_robot_direction(direction), real_modified_map)

In [159]:
map[current_position[0]][current_position[1]] = '@'
pretty_print_map(map)

##################################################
#..O.OOO.#.OO.OO#OOOOOO...OOO#OOO...OOOO....O....#
#.OO..OO#O....#OOOO#O.........OOOO....O.##.OO..O.#
#..OO#OOOO..OOOO...#O.................OOOOO#OO...#
###.#O....OO#O......#.......#O........O#OO.O..O..#
#OO.........OO..............#OO...#......OOOO.O..#
#OO....O....##..............O#O.OO.........OO.O#O#
#OO...#O...OO........O.#......O.O..........O..O#.#
#OO..@O#...O..O...............O.#..........OO...##
#OOO.....#.#............O...O.........OOO..OOOOO##
#O..............O........##..#OO#......O#OO....#O#
#O....O...#..........OOO.O....#O...O...#OOO....OO#
#......#..O...OOO#.#.#O..O...OOO#OOO.....OO....O.#
#..#O..#...#OOOOO.OOO..........####O.....O#..O#..#
#..#..OO.#.#OOOO#OO............O#OOO.........OO.O#
#.O#.....OOO##O#..O.O...#O..O.......OOO.O....#...#
#........O.#.O#O..#.O.....O#O......#O.O.O.#..#...#
#O...O.O.#....OO..O..#...O##....#..O#.#.OO.......#
#O.....O...O#....OO.OO.#OOOO..........OOOO....#..#
#O........OO#..#..O.....#OOO...

In [160]:
print(compute_gps_sum(map))

1478649


## Second star

In [170]:
def transform_map(original_map):
    """
    Transforms the original map by replacing each tile:
    - '#' -> '##'
    - 'O' -> '[]'
    - '.' -> '..'
    - '@' -> '@.'
    """
    # Define the replacement mapping
    replacement = {
        '#': ['#','#'],
        'O': ['[',']'],
        '.': ['.','.'],
        '@': ['@','.']
    }
    
    transformed_map = []
    
    for row in original_map:
        new_row = ''
        for cell in row:
            # Replace the cell according to the mapping
            new_cells = replacement.get(cell, cell)
            for new_cell in new_cells:
                new_row += new_cell
        transformed_map.append(new_row)
    
    transformed_map = [list(row) for row in transformed_map]
    return transformed_map

In [171]:
transformed_map = transform_map(example_map)
pretty_print_map(transformed_map)

####################
##....[]....[]..[]##
##............[]..##
##..[][]....[]..[]##
##....[]@.....[]..##
##[]##....[]......##
##[]....[]....[]..##
##..[][]..[]..[][]##
##........[]......##
####################


In [181]:
def move_robot_2(current_position, current_direction, map_grid):
    """
    Moves the robot on the transformed map based on the current direction.
    If the robot encounters a '[]', it attempts to push all aligned '[]'s in that direction.
    """
    def push_boxes(position, direction):
        """
        Attempts to push all aligned '[]' boxes in the given direction.
        """
        delta_row, delta_col = direction
        row, col = position
        boxes_to_push = []
        
        # Collect all consecutive '[]' boxes in the direction
        while True:
            target_row = row + delta_row
            target_col = col + delta_col
            # Check boundaries
            if target_row < 0 or target_row >= len(map_grid):
                return False
            if target_col < 0 or (target_col * 2 + 1) >= len(map_grid[0]):
                return False
            # Extract the box at the target position
            cell = ''.join(map_grid[target_row][target_col*2 : target_col*2+2])
            if cell == '[]':
                boxes_to_push.append((target_row, target_col))
                row, col = target_row, target_col
            else:
                break
        
        # Determine the next position after the last box
        next_row = row + delta_row
        next_col = col + delta_col
        
        # Check if the space after the last box is empty '..'
        if next_row < 0 or next_row >= len(map_grid):
            return False
        if next_col < 0 or (next_col * 2 + 1) >= len(map_grid[0]):
            return False
        next_cell = ''.join(map_grid[next_row][next_col*2 : next_col*2+2])
        if next_cell != '..':
            return False
        
        # Push all boxes starting from the last one to avoid overwriting
        for box_row, box_col in reversed(boxes_to_push):
            push_to_row = box_row + delta_row
            push_to_col = box_col + delta_col
            # Move the box
            map_grid[push_to_row][push_to_col*2] = '['
            map_grid[push_to_row][push_to_col*2 + 1] = ']'
            # Clear the box's original position
            map_grid[box_row][box_col*2] = '.'
            map_grid[box_row][box_col*2 + 1] = '.'
        
        return True
    
    # Calculate the new virtual tile position based on direction
    new_tile_row = current_position[0] + current_direction[0]
    new_tile_col = current_position[1] + current_direction[1]
    
    # Convert tile coordinates to map grid indices (character coordinates)
    new_map_row = new_tile_row
    new_map_col_start = new_tile_col
    new_map_col_end = new_map_col_start + 2
    
    # Check boundaries
    if new_tile_row < 0 or new_tile_row >= len(map_grid):
        # Movement out of bounds; treat as a wall
        print("Movement out of bounds. Cannot move.")
        return current_position, map_grid
    if new_map_col_start < 0 or new_map_col_end > len(map_grid[0]):
        # Movement out of bounds; treat as a wall
        print("Movement out of bounds. Cannot move.")
        return current_position, map_grid
    
    # Extract the cell at the target position
    new_virtual_value = ''.join(map_grid[new_map_row][new_map_col_start : new_map_col_end])
    
    if new_virtual_value == '##':
        # Cannot move into a wall
        print("Hit a wall. Cannot move.")
        return current_position, map_grid
    elif new_virtual_value == '[]':
        # Attempt to push the box(es) in the current direction
        can_push = push_boxes((new_tile_row, new_tile_col), current_direction)
        if can_push:
            # Move the robot into the box's previous position
            current_map_row = current_position[0]
            current_map_col_start = current_position[1] 
            current_map_col_end = current_map_col_start + 2
            # Clear the robot's current position
            map_grid[current_map_row][current_map_col_start] = '.'
            map_grid[current_map_row][current_map_col_start + 1] = '.'
            # Place the robot at the new position
            map_grid[new_map_row][new_map_col_start] = '@'
            map_grid[new_map_row][new_map_col_start + 1] = '.'
            # Update robot position
            current_position = (new_tile_row, new_tile_col)
            print(f"Robot moved to ({new_tile_row}, {new_tile_col}) and pushed the box(es).")
        else:
            # Push failed; robot cannot move
            print("Push failed. Cannot move.")
            return current_position, map_grid
    elif new_virtual_value == '..' or new_virtual_value == '@.':
        # Move robot into the empty space or onto '@.' (if robot is already there)
        current_map_row = current_position[0]
        current_map_col_start = current_position[1]
        current_map_col_end = current_map_col_start + 2
        # Clear the robot's current position
        map_grid[current_map_row][current_map_col_start] = '.'
        map_grid[current_map_row][current_map_col_start + 1] = '.'
        # Place the robot at the new position
        map_grid[new_map_row][new_map_col_start] = '@'
        map_grid[new_map_row][new_map_col_start + 1] = '.'
        # Update robot position
        current_position = (new_tile_row, new_tile_col)
        print(f"Robot moved to ({new_tile_row}, {new_tile_col}).")
    else:
        # Encountered an unexpected character; do nothing
        print(f"Encountered an unexpected cell '{new_virtual_value}'. Cannot move.")
        return current_position, map_grid
    
    return current_position, map_grid

In [182]:
transformed_modified_map = deepcopy(transformed_map)
starting_position = find_starting_position(transformed_modified_map)
current_position = starting_position
transformed_modified_map[current_position[0]][current_position[1]] = '.'
for direction in example_instructions:
    current_position, map = move_robot_2(current_position, find_robot_direction(direction), transformed_modified_map)

Encountered an unexpected cell '].'. Cannot move.
Robot moved to (5, 8).
Robot moved to (6, 8) and pushed the box(es).
Robot moved to (6, 9).
Encountered an unexpected cell '.['. Cannot move.
Encountered an unexpected cell '.@'. Cannot move.
Encountered an unexpected cell '.['. Cannot move.
Encountered an unexpected cell '.['. Cannot move.
Robot moved to (6, 10).
Push failed. Cannot move.
Robot moved to (6, 11).
Encountered an unexpected cell '].'. Cannot move.
Encountered an unexpected cell '].'. Cannot move.
Encountered an unexpected cell '].'. Cannot move.
Encountered an unexpected cell '].'. Cannot move.
Encountered an unexpected cell '].'. Cannot move.
Robot moved to (6, 12).
Robot moved to (7, 12).
Encountered an unexpected cell ']@'. Cannot move.
Encountered an unexpected cell '.['. Cannot move.
Robot moved to (8, 12).
Robot moved to (7, 12).
Robot moved to (8, 12).
Encountered an unexpected cell ']@'. Cannot move.
Hit a wall. Cannot move.
Encountered an unexpected cell ']@'. Ca

In [178]:
map[current_position[0]][current_position[1]] = '@'
pretty_print_map(map)

####################
##..............[]##
##................##
##................##
##................##
##..##............##
#@@...............##
##................##
##................##
####################
